In [4]:
import numpy as np

In [5]:
def readfile(filename):
    # open file
    f = open(filename,'r')
    
    # read header info line by line 

    # read first two lines FIRST 
    line = f.readline()
    label, value = line.split()
    time = float(value)

    line = f.readline()
    label, value = line.split()
    total = float(value)
    
    # close file
    f.close()

    # read the remainder of the file, skipping 
    #the first 3 lines and storing 
    #the values as arrays with the column headers 
    #given in line 4
    data = np.genfromtxt(filename, dtype=None, names=True, skip_header=3)
    
    # this will return the time of the snapshot, 
    #total number of particles 
    #and an array that stores the remainder of the data. 
    return time, total, data

In [6]:
def COMDef(x,y,z,m):
    # compute the center of mass 
    # note: since all particles have the same
    # mass when consider only one type
    # this is equivalently  np.sum(x)/len(x)
    
    # xcomponent
    cXP = np.sum(x*m)/np.sum(m)
    # ycomponent
    cYP = np.sum(y*m)/np.sum(m)
    # zcomponent
    cZP = np.sum(z*m)/np.sum(m)
    
    return cXP,cYP,cZP

In [16]:
# function to compute the POSITION and 
# VELOCITY of the center of mass of Galaxy X 
#using particles of Type Y at SnapNumber Z 

# input: String with Galaxy Name
# input: 1,2,3 indicating particle type
# input: Integer with Snapshot Number  e.g. 0,256,700
# input:  Delta = tolerance for the iteration
# input:  dec = factor by which to decrease the search radius
# Return: Array of  ( time, x, y, z) of center of mass 

def COM(galaxy, PType, Snap, delta,dec):

    # Read in the Right file, identified by "Galaxy" and "Snap"

     # add a string of the filenumber to the value "000" 
    ilbl = '000' + str(Snap)
        # remove all but the last 3 digits
    ilbl = ilbl[-3:]
        # create filenames 
    filename=galaxy + "_" + ilbl + '.txt'
    print filename
    
    # read in the filename 
    time, total, data = readfile(filename)
    
     # identify all particles of required type
    index = np.where(data['type']== float(PType))

    
    # LOAD positions of particles of required type

    # store x position of particles 
    xP = data['x'][index]
    # store y position of particles
    yP = data['y'][index]
    # store z position of particles 
    zP = data['z'][index]
     # store x position of particles 
    vxP = data['vx'][index]
    # store y position of particles
    vyP = data['vy'][index]
    # store z position of particles 
    vzP = data['vz'][index]
   
    # store mass of all particles of required type
    mP = data['m'][index]

    # compute the center of mass position using all
    # particles of given type
    cXP, cYP, cZP = COMDef(xP,yP,zP,mP)
    
    # compute the center of mass velocity using all 
    # particles of given type
    cVXP, cVYP, cVZP = COMDef(vxP,vyP,vzP,mP)
    
    ##############
    # FOR TESTING
    ##############
    # print the center of mass position (x,y,z) relative to 0,0,0
    # using all particles of given type (no position cuts)
    #print "Center of Mass Position", cXP, cYP, cZP
    # compute the magnitude
    #cRP = np.sqrt(cXP**2 + cYP**2 + cZP**2) 
    #print "Magnitude", cRP
 
    #print "Center of Mass Velocity", cVXP, cVYP, cVZP
    # compute the magnitude
    #cVP = np.sqrt(cVXP**2 + cVYP**2 + cVZP**2) 
    #print "Magnitude", cVP
    ####################
    
    
    #  start an iterative process to determine the center of mass 
      
    # compute difference between particle coordinates 
    # and first stab at COM
    xNew = xP - cXP
    yNew = yP - cYP
    zNew = zP - cZP
    rNew = np.sqrt(xNew**2.0 + yNew**2.0 +zNew**2.0)
    
    # find the max distance of the particle from COM
    # start at half that radius
    maxR2 = max(rNew)/2.0
    #print "MAX", max(rNew)
    
    
    # set an initial disparity for the COM
    diff = 100.0 
    
    # while the difference between COM positions are greater than delta
    # continue iteration by checking 1/2 radius
    while(diff > delta):
                
        # identify particles within some fraction of the max radius
        indexN = np.where(rNew < float(maxR2))
        
        # make sure there are enough particles for the COM
        # calculation to be meaningful
        # here set for 100 particles 
        if (len(indexN[0][:]) < 100):
            print "Too Small"
            break
        
        
        # compute COM using only those particles within maxR2
        cXPnew, cYPnew, cZPnew = COMDef(xP[indexN],yP[indexN],zP[indexN],mP[indexN])
        
        # compute the COM velocity for these particles within maxR2 
        cVXPnew, cVYPnew, cVZPnew = COMDef(vxP[indexN],vyP[indexN],vzP[indexN],mP[indexN])        
        
        # what is the difference between this new COM and the previous one
        # check difference in each component of the vector
        dX = np.abs(cXP - cXPnew)
        dY = np.abs(cYP - cYPnew)
        dZ = np.abs(cZP - cZPnew)
        dArray = [dX,dY,dZ]
        diff = np.max(dArray)
        #Could have also checked difference with the magnitude of the vector
        #diff = np.sqrt(dX**2 + dY**2 + dZ**2)

        # reset the scenario for the next iteration
        
        # we assume the new COM  is more accurate than the previous one 
        # so we save the new COM 
        cXP = cXPnew
        cYP = cYPnew
        cZP = cZPnew
        cVXP = cVXPnew
        cVYP = cVYPnew
        cVZP = cVZPnew
        
        # reset the difference between particle position and new COM
        xNew = xP - cXP
        yNew = yP - cYP
        zNew = zP - cZP
        
        # compute magnitude of distance from new COM
        rNew = np.sqrt(xNew**2.0 + yNew**2.0 +zNew**2.0)
        

        # TESTING THE PROCESS ABOVE
        ############################
        print "decreased maxR", maxR2
        print diff
        print (diff > delta)
        #############################
        
        # for next iteration start at an even smaller radius based on 
        # input, dec
        # it seems that 1/4 yields better results than dividing by 2. 
        maxR2 = max(rNew[indexN])/dec
        
        
    # store time and x,y,z  position of COM to an array
    COMP = [time, round(cXP),round(cYP),round(cZP),round(cVXP),round(cVYP),round(cVZP)]
    
    # print the magnitude of the COM 
    print "%s Center of Mass position using particle type %s is" %(galaxy,PType), round(np.sqrt(cXP**2+ cYP**2+ cZP**2)), "kpc"
    print "%s Center of Mass velocity using particle type %s is" %(galaxy,PType), round(np.sqrt(cVXP**2+ cVYP**2+ cVZP**2)), "km/s"
    print "%s Center of mass position vector using particle type %s" %(galaxy,PType), COMP[1],COMP[2],COMP[3]
    print "%s Center of mass velocity vector using particle type %s" %(galaxy,PType), COMP[4],COMP[5],COMP[6]
    
    # return the array with stored quantities 
    return COMP


In [21]:
# Question 5
# giving a tolerance of 2 
# radius decreases by 4 
print "MW HALO SNAP 0"
MW_Halo0= COM("MW",1,0,2,4)
print " "
print "MW DISK"
MW_Disk0= COM("MW",2,0,2,4)
print " "
print "MW Bulge"
MW_Bulge0= COM("MW",3,0,2,4)

MW HALO SNAP 0
MW_000.txt
decreased maxR 5536.67774223
2.11437138994
True
decreased maxR 1383.10205975
3.61691566484
True
decreased maxR 345.433318075
0.730824456454
False
MW Center of Mass position using particle type 1 is 1.0 kpc
MW Center of Mass velocity using particle type 1 is 2.0 km/s
MW Center of mass position vector using particle type 1 -1.0 0.0 -0.0
MW Center of mass velocity vector using particle type 1 -0.0 2.0 -1.0
 
MW DISK
MW_000.txt
decreased maxR 22.0409626668
0.378155500481
False
MW Center of Mass position using particle type 2 is 3.0 kpc
MW Center of Mass velocity using particle type 2 is 3.0 km/s
MW Center of mass position vector using particle type 2 -1.0 3.0 -1.0
MW Center of mass velocity vector using particle type 2 -0.0 2.0 -1.0
 
MW Bulge
MW_000.txt
decreased maxR 500.79209968
0.158834391679
False
MW Center of Mass position using particle type 3 is 4.0 kpc
MW Center of Mass velocity using particle type 3 is 10.0 km/s
MW Center of mass position vector using pa